<a href="https://colab.research.google.com/github/AshSlake/NPC-Studio-Com-IA-Gemini/blob/main/Projeto_NPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NPC-STUDIO**


In [32]:
!pip install -q -U google-generativeai
!pip install markdown

In [33]:
# Import the Python SDK
import google.generativeai as npc_g
from google.colab import userdata
api_key=userdata.get('KEY')
npc_g.configure(api_key=api_key)

In [34]:
configuracao_de_geracao = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [35]:
safety_settings={
    'HATE' : 'BLOCK_NONE',
    'HARASSMENT' : 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

**Vamos inicializar o modelo**

In [36]:
model = npc_g.GenerativeModel(model_name="gemini-1.0-pro-latest",
                              generation_config=configuracao_de_geracao,
                              safety_settings=safety_settings,)

**Agora vamos comecara a desenvolver seu npc**

In [ ]:
import random
import markdown
import os  # --> Importe o módulo 'os' para trabalhar com diretórios

def gerar_npc():
    """Coleta informações do usuário e gera uma ficha de NPC."""

    # Coleta informações básicas
    print("Escolha o sexo do NPC:")  # --> Apresenta as opções de sexo
    print("1. Masculino")
    print("2. Feminino")
    print("3. Não binário")
    print("4. Outro")

    while True: # --> Loop para garantir uma escolha válida
        try:
            opcao_sexo = int(input("Digite o número da opção desejada: "))  # --> Recebe a escolha do usuário
            if 1 <= opcao_sexo <= 4:
                break
            else:
                print("Opção inválida. Digite um número entre 1 e 4.")
        except ValueError:
            print("Entrada inválida. Digite um número.")

    sexo = {  # --> Dicionário para traduzir a escolha em string
        1: "masculino",
        2: "feminino",
        3: "não binário",
        4: "outro"
    }[opcao_sexo]

    while True:  # --> Loop para garantir uma idade válida (número)
        try:
            idade = int(input("Idade do NPC (digite um número): "))
            idade = str(idade) # --> Converte para string para usar na ficha
            break
        except ValueError:
            print("Entrada inválida. Digite apenas números para a idade.")

    nome = input("Nome do NPC: ")
    habilidade = input("Habilidade especial? (sim/não): ")

    # Gera informações ausentes
    if not nome:
        prompt = f"Sugira um nome {sexo.lower()}."
        response = model.generate_content(prompt)
        nomes_sugeridos = response.text.split("\n")
        nome = random.choice(nomes_sugeridos)
    if habilidade.lower() != "sim":
        habilidade = "Não possui habilidades especiais"
    else:
        habilidade = input(f"Qual é a habilidade de {nome.lower()}? ")

    # Informações adicionais (opcional)
    adicionar_mais = input("Adicionar mais informações? (sim/não): ")
    informacoes_adicionais = input("Digite as informações adicionais: ") if adicionar_mais.lower() == "sim" else ""

    # Momento marcante (opcional)
    momento_marcante = ""
    if adicionar_mais.lower() != "sim":
        quer_momento_marcante = input("Deseja gerar um momento marcante para o NPC? (sim/não): ")  # --> Pergunta ao usuário
        if quer_momento_marcante.lower() == "sim":
            prompt = f"Crie um momento marcante na vida de {nome.lower()}."
            response = model.generate_content(prompt)
            momento_marcante = response.text

    # Tipo de RPG
    tipo_rpg = input("Tipo de RPG (fantasia/sci-fi/moderno/outro): ")

    # Gera descrição e história
    descricao_npc = f"{nome} tem {idade} anos e possui a habilidade de {habilidade}."
    prompt_historia = f"Escreva um breve resumo da história de {nome}, um(a) {sexo} de {idade} anos com a habilidade de {habilidade}, em um mundo de {tipo_rpg}. Limite a resposta a no máximo 100 tokens."
    response = model.generate_content(prompt_historia)
    resumo_historia = response.text

    # Chama a função para gerar a ficha em Markdown
    ficha_markdown = gerar_ficha_markdown(nome, sexo, idade, habilidade,
                                       informacoes_adicionais, momento_marcante,
                                       descricao_npc, resumo_historia)

    # Converte a ficha Markdown para HTML
    ficha_html = markdown.markdown(ficha_markdown)

    # Imprime o nome do NPC em vermelho e a ficha HTML

    print("\033[1m" + "FICHA DO NPC".center(40, '=') + "\033[0m") # Título em negrito
    print(f"Nome: \033[92m{nome}\033[0m")  # Nome em verde
    print(f"Sexo: {sexo}")
    print(f"Idade: {idade} anos")
    print(f"Habilidade: {habilidade}")
    if informacoes_adicionais:
     print(f"Informações Adicionais: {informacoes_adicionais}")
    print(f"Momento Marcante: {momento_marcante}")
    print(f"Descrição: {descricao_npc}")
    print(f"História: {resumo_historia}\n")

    print("-------------")
    print("-------------")

    # Pergunta ao usuário se deseja salvar a ficha
    salvar_ficha = input("Deseja salvar a ficha do NPC? (sim/não): ")
    if salvar_ficha.lower() == "sim":
        nome_arquivo_sugerido = f"{nome.upper()}_FICHA.md"  # Sugere um nome de arquivo
        nome_arquivo = input(f"Digite o nome do arquivo (sugestão: {nome_arquivo_sugerido}): ")

        # Cria o subdiretório 'fichas' se ele não existir
        os.makedirs("fichas", exist_ok=True)

        # Salva a ficha em um arquivo .md no subdiretório 'fichas'
        caminho_arquivo = os.path.join("fichas", nome_arquivo)
        with open(caminho_arquivo, "w") as arquivo_md:
            arquivo_md.write(ficha_markdown)
        print(f"Ficha salva como: {caminho_arquivo}")

    # Converte a ficha Markdown para HTML
    ficha_html = markdown.markdown(ficha_markdown)

    print("-------------")
    print("-------------")

    print("html para usar  no StackEdit")
    print(ficha_html)

def gerar_ficha_markdown(nome, sexo, idade, habilidade, informacoes_adicionais, momento_marcante, descricao_npc, resumo_historia):
    """Gera uma string com a ficha do NPC em formato Markdown."""
    ficha_markdown = f"""
---
# Ficha do NPC

**Nome:** {nome}
**Sexo:** {sexo}
**Idade:** {idade} anos
**Habilidade:** {habilidade}

"""
    if informacoes_adicionais:
        ficha_markdown += f"**Informações Adicionais:** {informacoes_adicionais}\n\n"
    ficha_markdown += f"""
**Momento Marcante:** {momento_marcante}

**Descrição:** {descricao_npc}

**História:** {resumo_historia}
---
"""
    return ficha_markdown  # --> Indentação correta aqui

gerar_npc()

